<a href="https://colab.research.google.com/github/amandatz/computational-linear-algebra/blob/main/Atividade2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade 2

In [1]:
using LinearAlgebra

## Questão 1

## Questão 2

## Questão 3

## Questão 4